In [ ]:
# 구글 드라이브 연결

from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

project_path = '/content/drive/MyDrive/examples/2장/data'
os.makedirs(project_path, exist_ok=True)

# 현재 경로를 /content/drive로 설정
os.chdir(project_path)
print("현재 작업 디렉토리:", os.getcwd())
os.listdir(project_path)

Mounted at /content/drive
현재 작업 디렉토리: /content/drive/MyDrive/ch2


['shipsnet',
 'shipsnet.7z',
 'shipsnet.json',
 'dance.mov',
 'heart_rgb.png',
 'heart_gray.png',
 'heart.png',
 'sudoku.jpeg',
 'dog.png',
 'cat.png',
 'models',
 'coco']

In [ ]:
# Darknet 설치 (yolo 모델 구현체)
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!make

Cloning into 'darknet'...
remote: Enumerating objects: 15851, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15851 (delta 5), reused 13 (delta 4), pack-reused 15833
Receiving objects: 100% (15851/15851), 14.42 MiB | 10.10 MiB/s, done.
Resolving deltas: 100% (10671/10671), done.
Updating files: 100% (2056/2056), done.
/content/drive/MyDrive/ch2/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused varia

In [ ]:
# YOLOv3 weights 다운로드
!wget https://pjreddie.com/media/files/yolov3.weights

# 설정 파일 및 클래스 이름 다운로드
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov3.cfg
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/data/coco.names

--2024-06-19 13:33:01--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  30.6MB/s    in 7.5s    

2024-06-19 13:34:25 (31.4 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2024-06-19 13:34:25--  https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0.001s  

2

In [ ]:
import cv2
import numpy as np
import time

# 네트워크 및 가중치 파일 경로
config_path = 'yolov3.cfg'
weights_path = 'yolov3.weights'
names_path = 'coco.names'

# 클래스 라벨 로드
with open(names_path, 'r') as f:
    classes = f.read().strip().split('\n')

# 네트워크 설정
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]


In [ ]:
import matplotlib.pyplot as plt

# 비디오 파일 경로 (업로드한 파일 이름으로 변경)
video_path = os.path.join(project_path, 'dance.mov')

# 비디오 캡처 초기화
cap = cv2.VideoCapture(video_path)

# 비디오 쓰기 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_path = os.path.join(project_path, 'output.avi')

out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

In [ ]:
# 비디오 프레임 처리 및 저장
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임의 블롭 생성
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # 예측 결과 분석
    class_ids = []
    confidences = []
    boxes = []
    for o in outs:
        for detection in o:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # 결과 프레임에 그리기
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    out.write(frame)

cap.release()
out.release()

print("저장 완료: output.avi")

저장 완료: output.avi
